In [16]:
import pandas as pd
import json
from pprint import pprint
import os
import json
# from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
from IPython.display import display, Markdown
from dotenv import load_dotenv, find_dotenv
from langchain.document_loaders import TextLoader, CSVLoader, JSONLoader, PyPDFLoader
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate

In [17]:
#replace with your own env file containing API keys
load_dotenv(find_dotenv('Resources\keys.env'))
huggingfacehubapi = os.getenv('HuggingfaceRead')
# print(huggingfacehubapi)


In [18]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

## Create the function to load the vector database and chat.  Using HuggingFaceEndpoint mistralai/Mistral-7B-Instruct-v0.1 as the LLM . 
## Create a question answer retreival chain from langchain.chains framework

In [19]:
import warnings 
faiss_index_path = "Resources/vector"  
warnings.filterwarnings(action="ignore")
def load_faiss_and_chat(faiss_index_path=faiss_index_path):
    """Loads FAISS and creates a chatbot using Hugging Face LLM."""
    
    # Load FAISS vector store
    vector_store = FAISS.load_local(faiss_index_path, embedding_model,allow_dangerous_deserialization=True)

    
    llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    temperature=0.7,
    max_new_tokens=512,
    huggingfacehub_api_token=huggingfacehubapi)


    # # Custom prompt template
    # prompt_template = PromptTemplate(
    #     input_variables=["context", "question"],
    #     template="""You are an intelligent assistant answering questions based on the context below. 
    #     If the answer is not in the context, say you don't know. 
    #     Use a tone like you're chatting with someone in a store. 
    #     Context:{context}
    #     Question: {question}
    #     Answer:""")

    # Custom prompt template
    prompt_template = PromptTemplate(
        input_variables=["context", "question","chat_history"],
        template="""You are a helpful store assistant helping a customer based on the reviews and product info below. 
        If the answer is not in the context, say "I don't know." 
        When making recommendations, provide only the top 3 distinct products in the following format: 
        Make sure to return the top 3 distinct recommendations. If there are duplicates, choose only one entry for each product.
        **Recommendations**
        1. [Product Title] , [price] , [color]
            **Review**:[short summary from reviews]
        Do not display the raw product data or context. Focus on generating helpful recommendations in a conversational tone.
        Chat History:
        {chat_history}

        Question: {question}
        Context:{context}
        Answer:""")

    memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)

    qa_chain = ConversationalRetrievalChain.from_llm(llm=llm,
                                                     retriever=vector_store.as_retriever(),
                                                     memory=memory,
                                                     combine_docs_chain_kwargs={"prompt" : prompt_template})


    # # Create a QA chain using retrieval
    # qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vector_store.as_retriever(),
    #                                         chain_type_kwargs={"prompt": prompt_template},
    #                                         return_source_documents=False)

    while True:
        memory.clear()
        query = input("\nAsk a question (or type 'exit' to quit),'reset' to clear memory: ")
        if query.lower() == "exit":         
            print("Goodbye!")
            break
        elif query.lower() == "reset":
            memory.clear()
            print("Memory has been cleared")
            continue
        response = qa_chain.invoke(query)
        print(f"\n Chatbot: {response['answer']}") 
    



In [20]:
load_faiss_and_chat()


 Chatbot:  Hello! How may I assist you today?

 Chatbot: 
        1. BJPKPK 2 pcs Insulated Coffee Mug, 14oz Insulated Coffee Mug with Lid, Stainless Steel Insulated Coffee Mug with Splash Proof Lid-White. Price: $16.99.Rating: 5 stars. Color: White-14oz.
            **Review**: The mugs are great at keeping my coffee piping hot and the color is bright.
        2. Coffee Travel Mug, 14OZ Double Walled Insulated Vacuum Coffee Tumbler With Leakproof Flip Insulated Coffee Mug, For Hot And Cold Water Coffee And Tea In Travel Car Office School Camping (Light Green). Price: $14.99.Rating: 4 stars. Color: Light Green.
            **Review**: The mug keeps coffee warm nearly all day and fits cupholders.
        3. U.S. Navy Customizable Rank Insignia Ceramic Coffee/Cocoa Mug by Carpe Diem Designs, Made in the U.S.A.. Price: $17.95.Rating: 5 stars. Color: E4.
            **Review**: The cup is well made and looks good, and it has a good size and value.

 Chatbot: 
        I'm sorry to hear tha